In [ ]:
!pip uninstall -y numpy
!pip install numpy==1.26.4 --no-cache-dir --force-reinstall
import os
os.kill(os.getpid(), 9)  # Isso reinicia o kernel do Colab


In [ ]:
!pip install rdkit-pypi


# Dataset conjunto

In [ ]:
import pandas as pd

df = pd.read_csv('/content/Dataset Both.csv')
df.shape

In [ ]:
from rdkit import Chem

def smiles_valido(smiles):
    """
    Verifica se um SMILES é válido.

    Parâmetros:
        smiles (str): String do SMILES.

    Retorna:
        bool: True se o SMILES é válido, False caso contrário.
    """
    if not isinstance(smiles, str) or smiles.strip() == "":
        return False
    try:
        mol = Chem.MolFromSmiles(smiles)
        return mol is not None
    except:
        return False


In [ ]:
df['SMILES_valido'] = df['SMILES'].apply(smiles_valido)
df['SMILES_valido'].value_counts()

In [ ]:
df.drop(columns=['Chemical', 'Identificador', 'SMILES_valido'], inplace=True)
df.shape

In [ ]:
df.drop_duplicates(inplace=True)
df.head()
df.shape

In [ ]:
df['Results'] = df['Results'].str.lower()
df['Type'] = df['Type'].str.lower()

In [ ]:
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
# Mapear os valores para 1 (positivos) e 0 (negativos)
mapping = {"positive": 1, "negative": 0, "ambiguous": None, "inconclusive": None}
df["Results"] = df["Results"].map(mapping)

# Remover valores nulos
df = df.dropna()

In [ ]:
df.shape

In [ ]:
# --- Passo 1 ---
# Para cada SMILES, identificar quais possuem mais de um tipo
smiles_mult_type = df.groupby("SMILES")["Type"].nunique()
smiles_mult_type = smiles_mult_type[smiles_mult_type > 1].index

# Para os SMILES que possuem mais de um 'Type', manter somente as linhas onde 'Type' é 'in vivo'
df_filtrado = df[~(df["SMILES"].isin(smiles_mult_type) & (df["Type"] != "in vivo"))]

# Remover duplicatas, se existirem
df_filtrado = df_filtrado.drop_duplicates()

In [ ]:
df_filtrado.shape

In [ ]:
# --- Passo 2 ---
# Agora, identificar quais SMILES (no df filtrado) possuem mais de um 'Results'
smiles_mult_result = df_filtrado.groupby("SMILES")["Results"].nunique()
smiles_mult_result = smiles_mult_result[smiles_mult_result > 1].index

# Remover os SMILES que possuem mais de um resultado
df_final = df_filtrado[~df_filtrado["SMILES"].isin(smiles_mult_result)]

In [ ]:
df_final['Results'].value_counts()

In [ ]:
df_final = df_final.drop(columns='Type')

In [ ]:
df_final.to_csv('df_final.csv', index=False)

# Dataset Vivo

In [ ]:
df_vivo= pd.read_csv('/content/Dataset in Vivo.csv')
df_vivo.shape

In [ ]:
# Mapear os valores para 1 (positivos) e 0 (negativos)
mapping = {"positive": 1, "negative": 0, "ambiguous": None, "inconclusive": None}
df_vivo["Results"] = df_vivo["Results"].map(mapping)

# Remover valores nulos
df_vivo = df_vivo.dropna()
df.shape

In [ ]:
# Agora, identificar quais SMILES (no df filtrado) possuem mais de um 'Results'
smiles_mult_result = df_vivo.groupby("SMILES")["Results"].nunique()
smiles_mult_result = smiles_mult_result[smiles_mult_result > 1].index

# Remover os SMILES que possuem mais de um resultado
df_final_vivo = df_vivo[~df_vivo["SMILES"].isin(smiles_mult_result)]
df_final_vivo.shape

In [ ]:
df_final_vivo.to_csv('df_final_vivo.csv', index=False)

# Dataset Vitro

In [ ]:
df_vitro= pd.read_csv('/content/Dataset in vitro.csv')
df.shape

In [ ]:
df_vitro.drop(columns= ['Chemical','Identificador'],inplace=True)
df_vitro['Results'] = df_vitro['Results'].str.lower()

In [ ]:
# Mapear os valores para 1 (positivos) e 0 (negativos)
mapping = {"positive": 1, "negative": 0, "ambiguous": None, "inconclusive": None}
df_vitro["Results"] = df_vitro["Results"].map(mapping)

# Remover valores nulos
df_vitro = df_vitro.dropna()
df_vitro.shape

In [ ]:
# Agora, identificar quais SMILES (no df filtrado) possuem mais de um 'Results'
smiles_mult_result = df_vitro.groupby("SMILES")["Results"].nunique()
smiles_mult_result = smiles_mult_result[smiles_mult_result > 1].index

# Remover os SMILES que possuem mais de um resultado
df_final_vitro = df_vitro[~df_vitro["SMILES"].isin(smiles_mult_result)]
df_final_vitro.shape

In [ ]:
df_final_vitro.to_csv('df_final_vitro.csv', index=False)